In [1]:
%load_ext autoreload
%autoreload 2

import sys

# For managing relative imports from notebook
if '..' not in sys.path: sys.path.append('..')

import config.config as dfc
import deepfake.dfutillib as df

# Dataset Downloader

This Notebook is just for downloading the full dataset off of the kaggle website. You can do this by hand from your browser  
locally, I really wrote it for AWS to pull directly onto an EC2 or Sagemaker instance (but works for local machine too). 

This uses **wget** and requires a **cookies.txt** file (stoopid kaggle CLI wouldn't work for the real deal dataset of this  
competition, and I couldn't get requests to work either because there's an authentication step, it's not fully public URIs).

To get the cookies.txt file: login to the kaggle site (you'll probably need to join the competition too), then use this Chrome  
plugin to save the cookies.txt file: https://chrome.google.com/webstore/detail/cookiestxt/njabckikapfpffapmjgojcnbfjonfjfg?hl=en

**TODO:** will probably need to write code in here to push the dataset to S3 after download, unless just parking it on the  
EC2/Sagemaker disk ends up being cost effective enough or necessary for efficient enough training.

In [2]:
# NOTE: notebook requires appropriate setup of deepfake.config 
# Be prepared for 500GB to get downloaded into your DATA_LAKE
# (and you'll prob need another 500GB there to preprocess 
# it all too, but that doesn't happen from this notebook).

data_lake, cookies_txt = dfc.DATA_LAKE, dfc.COOKIES_TXT
assert df.file_exists(cookies_txt), f"Cookies file: {cookies_txt} does not exist."

In [3]:
print(data_lake)

/home/ec2-user/SageMaker/ebs/deepfake-sample-test


In [4]:
# Verify your setup with this little cell. Just pulls a single sample MP4
# file into your DATA_LAKE, unzips it and then removes the archive file.

zipname = 'aassnaulhq.mp4.zip'
url = ('https://www.kaggle.com/c/deepfake-detection-challenge/download/'
       'Z1QcG4KHiVQaMucH9dng%2Fversions%2FvI13Sc4R0yAI121wogYv%2F'
       'directories%2Ftest_videos%2Ffiles%2Faassnaulhq.mp4')

!wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
!unzip -q '{data_lake}/{zipname}' -d '{data_lake}'
!rm '{data_lake}/{zipname}'

--2020-03-30 03:18:45--  https://www.kaggle.com/c/deepfake-detection-challenge/download/Z1QcG4KHiVQaMucH9dng%2Fversions%2FvI13Sc4R0yAI121wogYv%2Fdirectories%2Ftest_videos%2Ffiles%2Faassnaulhq.mp4
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/16880/858837/compressed/test_videos/aassnaulhq.mp4.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1585797525&Signature=XuoQhHRpR%2F46UJq%2B7XjXgiEoWyusqoyd6g%2BzAUb74CKLzKeSOvRVzkG4pUje5MhPxLlmRgWdyOpDLnHtSPiAQ6emGYKITa6mmXbuNqIW85O8dmZeIJEfUqDbQ%2FjdWcHAbS3bEtXcxCumvFAz%2FN1wi3nfe2fvzgL2yQY4YaWy7E7fEjDds8fPPDskEC1OAOeDJCYTTk2ce%2BfrSO8iOv%2F4Cdgcx99L4gbiNuNlJ7%2BP3RXu%2FfdukLVZO3jsqCiecFeU52Mb8OMybrLYNRkjabBK3CLZKU8Yv%2BDAniPDj%2F7q%2B7D4xkEDKaHOi4aUjxKD2gHDpkpbJkjbK7nXxZwKWo69mg%3D%3D&response-content-disp

In [8]:
def download_kaggle_huge_zip():
    zipname = f'dfdc_train_all.zip'
    url = f'https://www.kaggle.com/c/16880/datadownload/{zipname}'
    !wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
    !unzip -q '{data_lake}/{zipname}' -d '{data_lake}'
    !rm '{data_lake}/{zipname}'

download_kaggle_huge_zip()

--2020-03-28 04:32:36--  https://www.kaggle.com/c/16880/datadownload/dfdc_train_all.zip
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1585629156&Signature=WqItMzXRm4Xq4IoIy1IzUxdZBCGwO5TgTjKoAW75sFDiXosu4xUih0nxJGAzO7DDrKHL5F9nIBJsEdeAQVm3ooJ5fft4HTdgqY8Z2AiBUstsBF4KPnIYZJQOGnXxedScGlqBXUhURCO57snrSYj9QT%2FsNCqDl1tqMSTmyGYyjpDYsh%2BjnlS6tZmp8K7v9VrYzO39wJ%2FWUDQDNDpxM5wGLXY58MMkH1DKYHHo8BEMlPscg8DEnZGQqHf%2FVhw7r0pzGcP%2BpWRPiK%2BbemOXfj1vDv%2FwBQ4oFoYA7kGcuRgoJiOMyCciaY1C%2BFtx8VfVNn9vKrLt%2BKVxxtwOGPBGk%2Fdsqw%3D%3D [following]
--2020-03-28 04:32:36--  https://storage.googleapis.com/kaggle-competitions-detached-data/16880/dfdc_train_all.zip?GoogleAccessId=web-data@kaggle-161607

In [5]:
def download_kaggle_test_zip():
    zipname = f'deepfake-detect.zip'
    url = 'https://www.kaggle.com/c/16880/download-all'
    !wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
    !unzip -q '{data_lake}/{zipname}' -d '{data_lake}'
    !rm '{data_lake}/{zipname}'

download_kaggle_test_zip()



--2020-03-30 03:25:16--  https://www.kaggle.com/c/16880/download-all
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/16880/858837/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1585797917&Signature=R4qymGa6dLQJ4qkr44utCZ7GWyfWgPs5bCQZp4yWdpG8iJXLui1G8RjEDj%2FHCUKSpCfi1BZkMS5aFq%2FiMagNb8%2BLLqdO9SUsOMK3BAg3%2Boeu77lC%2FIiMcF2U%2BuhHRVDOgpvWUZG%2BVRGGaTpMdd2FpbVYoHP4YbMKEb8IBm6FIQ1tK2J0mXIkaIDH%2BsJGRZhSkBCzmM15U63dm5xbD9%2FXtCdlJaQQfhuMF4jc81a%2BKcGM8bU4rgmEcpXF1kv%2BuyrcDFsXUG2xlWLdg00tL7639YqsIiyqsjmRLrZs9HfKjzgFZyYmt%2FWtrBmR0EKVOHrjsIjJHO8wSfeoaI4bIVZRNQ%3D%3D&response-content-disposition=attachment%3B+filename%3Ddeepfake-detection-challenge.zip [following]
--2020-03-30 03:25:17--  https://storage.googleapis.com/kaggle-competiti

In [17]:
%%bash

data_lake=/home/ec2-user/SageMaker/ebs/deepfake-detect-datalake

for zipname in $data_lake/*.zip; do
    unzip -q $zipname -d $data_lake &
done

#}

In [ ]:
# Download and extract (all 50) deepfake video file partitions.
# Note: kaggle urls and filenames are effectively hardcoded.

# Can set istart and istop (same indexing as range fcn) to pull a
# subsets of the partitions if something goes wrong in the middle.
def download_kaggle_data(istart=50, istop=None):
#{
    if istop is None: 
        istart, istop = 0, istart
        
    for i in range(istart, istop):
        zipname = f'dfdc_train_part_{i:02d}.zip'
        url = f'https://www.kaggle.com/c/16880/datadownload/{zipname}'
        !wget --load-cookies '{cookies_txt}' -O '{data_lake}/{zipname}' '{url}'
        !unzip -q '{data_lake}/{zipname}' -d '{data_lake}'
        !rm '{data_lake}/{zipname}'
#}

# NOTE: uncomment this line to download all partitions
download_kaggle_data()

# These two partitions, 18 and 35, are slightly incomplete relative to 
# metadata files, but re-downloading then did not correct the issue.
# download_kaggle_data(18, 19)
# download_kaggle_data(35, 36)